In [9]:
import openai
import time

In [3]:
openai.api_key = 'API_KEY'

### Simple Request

In [3]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Do you know the ultimatum game?"},
    ]
)

print(response)

{
  "id": "chatcmpl-8G5cFAIzNk50tXIBECFDygO1CsDcu",
  "object": "chat.completion",
  "created": 1698845527,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Yes, I am familiar with the ultimatum game. It is a widely studied and well-known economic game used in experimental economics. The ultimatum game involves two players: a proposer and a responder. The proposer is given a sum of money and is required to offer a portion of it to the responder. The responder can either accept or reject the offer. If the responder accepts, both players receive the agreed-upon amounts. However, if the responder rejects the offer, neither player receives any money. The game is interesting because it explores concepts of fairness, equity, and rational decision-making."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 16,
    "completion_tokens": 122,
    "total_tokens": 138
  }
}


### Ultimatium Game - Sample Request

In [4]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", 
         "content": """
         In the following scenario, Person B had to decide whether
         to accept or reject the proposal.
         Scenario: Person A is given $10. Person A will
         propose how to split the money between himself and Person B. 
         Then Person B will decide whether to accept
         or reject Person A’s proposal. If Person B accepts,
         then Person A and Person B get the money as they
         agreed to split. If Person B rejects, then Person A and
         Person B both receive nothing. Person A takes $6 for
         himself and offers Person B $4.
         Answer: Person B decides to
         """},
    ]
)

print(response)

{
  "id": "chatcmpl-8G6GV1Hs3QHewNWfu2J3yNTY931iZ",
  "object": "chat.completion",
  "created": 1698848023,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "reject Person A's proposal."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 140,
    "completion_tokens": 6,
    "total_tokens": 146
  }
}


### Ultimatum Game - 50 Iterations

mit Error, hat aber zumindest einmal funktioniert

In [21]:
answers = []
for _ in range(50):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", 
             "content": """
             In the following scenario, Person B had to decide whether
             to accept or reject the proposal.
             Scenario: Person A is given $10. Person A will
             propose how to split the money between himself and Person B. 
             Then Person B will decide whether to accept
             or reject Person A’s proposal. If Person B accepts,
             then Person A and Person B get the money as they
             agreed to split. If Person B rejects, then Person A and
             Person B both receive nothing. Person A takes $5.50 for
             himself and offers Person B $4.50.
             Answer: Person B decides to
             """},
        ],
        max_tokens=1
    )
    
    # Store the answer in the list
    answer = response['choices'][0]['message']['content']
    answers.append(answer.strip())
    
    # Add a 20-second delay before the next API call
    time.sleep(20)

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

In [22]:
answers

['reject',
 'reject',
 'reject',
 'reject',
 'accept',
 'reject',
 'reject',
 'reject',
 'reject',
 'reject',
 'reject',
 'reject',
 'reject',
 'reject',
 'reject',
 'accept',
 'accept',
 'reject',
 'accept',
 'reject',
 'reject',
 'reject',
 'accept',
 'reject',
 'reject',
 'reject',
 'reject',
 'reject',
 'reject',
 'accept']

Das kam raus als kein Error war:

In [16]:
# Count the number of "accepts"
accept_count = answers.count("accept")

# Calculate the share of "accepts"
accept_share = accept_count / len(answers)

# Accepts share for offer A $6 and B $4
accept_share

0.14

Hier habe ich noch GPT2 benutzt, kam aber keine sinnvolle Antwort raus.

In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [22]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [30]:
sentence = '''
    In the following scenario, Ms. Huang had to decide whether
    to accept or reject the proposal.
    Scenario: Mr. Wagner is given $10. Mr. Wagner will
    propose how to split the money between himself and Ms.
    Huang. Then Ms. Huang will decide whether to accept
    or reject Mr. Wagner’s proposal. If Ms. Huang accepts,
    then Mr. Wagner and Ms. Huang get the money as they
    agreed to split. If Ms. Huang rejects, then Mr. Wagner and
    Ms. Huang both receive nothing. Mr. Wagner takes $6 for
    himself and offers Ms. Huang $4.
    Answer: Ms. Huang decides to
    '''
input_ids = tokenizer.encode(sentence, return_tensors='pt')

In [36]:
output = model.generate(input_ids, max_length=190, num_beams=5, early_stopping=True)
tokenizer.decode(output[0], skip_special_tokens=True)

"\n    In the following scenario, Ms. Huang had to decide whether\n    to accept or reject the proposal.\n    Scenario: Mr. Wagner is given $10. Mr. Wagner will\n    propose how to split the money between himself and Ms.\n    Huang. Then Ms. Huang will decide whether to accept\n    or reject Mr. Wagner’s proposal. If Ms. Huang accepts,\n    then Mr. Wagner and Ms. Huang get the money as they\n    agreed to split. If Ms. Huang rejects, then Mr. Wagner and\n    Ms. Huang both receive nothing. Mr. Wagner takes $6 for\n    himself and offers Ms. Huang $4.\n    Answer: Ms. Huang decides to\n     accept or reject Mr. Wagner's proposal. Ms."